<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Importando Librerias</strong></h4>
</div>

In [1]:
import pandas as pd

xSubDataSet =  '../data/processed/SubDataSet.parquet'
xDataSet    =  '../data/processed/DataSet.parquet'
xTrain      =  '../data/processed/Train.parquet'
xValid      =  '../data/processed/Validation.parquet'
xMlFlow     =  'http://127.0.0.1:8080'
xPorcen     = 0.20

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Importando DataSet</strong></h4>
</div>

In [2]:
df = pd.read_parquet(xDataSet)
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,target
0,18393.0,2,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0
1,20228.0,1,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0,1.0
2,18857.0,1,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0,1.0
3,17623.0,2,169.0,82.0,150.0,100.0,1.0,1.0,0.0,0.0,1.0,1.0
4,17474.0,1,156.0,56.0,100.0,60.0,1.0,1.0,0.0,0.0,0.0,0.0


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Obteniendo Data Train y Test</strong></h4>
</div>

In [3]:
Valid_0 = df[df['target'] == 0].sample( frac = xPorcen )
Valid_1 = df[df['target'] == 1].sample( frac = xPorcen )

Valid = pd.concat([Valid_0, Valid_1])
Train = df.drop(Valid.index)

try:
    Valid.to_parquet(xValid)
    print(f"✅ El archivo \033[94m{xValid}\033[0m fue creado con Exito !!!")
    Train.to_parquet(xTrain)
    print(f"✅ El archivo \033[94m{xTrain}\033[0m fue creado con Exito !!!")
except Exception as e:
    print(f"❌ {type(e).__name__}: {e}")
    sys.exit(2)

del Valid_0, Valid_1, Valid, Train

✅ El archivo ../data/processed/Validation.parquet fue creado con Exito !!!
✅ El archivo ../data/processed/Train.parquet fue creado con Exito !!!


<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Creando Transformacion de Variables Categoricas a Numericas</strong></h4>
</div>

In [4]:
train = pd.read_parquet(xTrain)

variables_categoricas = train.select_dtypes(include=["object","category"]).columns.tolist()
print("Columnas categoricas: ",variables_categoricas) 

Columnas categoricas:  ['gender']


In [5]:
from category_encoders import CatBoostEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers = [ ('target_enc', TargetEncoder(), variables_categoricas) ],
    remainder='passthrough'  # deja las columnas numéricas como están
)

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">✅ <strong>Preparando Data para el Training</strong></h4>
</div>

In [6]:
X_train = train.drop('target', axis=1)
y_train = train['target']

test = pd.read_parquet(xValid)
X_test = test.drop('target', axis=1)
y_test = test['target']

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">
    ✅ <strong>Entrenando Modelos 
      <span style="color:#2e8b57; font-weight:bold; font-size:14px; background-color:#e0f8e0; padding:2px 4px; border-radius:3px;">
        sklearn.ensemble.RandomForestClassifier
      </span>
    </strong>
  </h4>
</div>


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.pipeline import Pipeline

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec
from mlflow.models import infer_signature

from datetime import datetime
import json

mlflow.set_tracking_uri(xMlFlow)

In [8]:

configuraciones = [
    # Profundo, muchos árboles, muestreo total
    {'n_estimators': 400, 'max_depth': None, 'min_samples_split': 2,  'min_samples_leaf': 1,
     'max_features': 'sqrt', 'bootstrap': True,  'criterion': 'gini',
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Profundidad media, regularización en hojas
    {'n_estimators': 250, 'max_depth': 18,  'min_samples_split': 4,  'min_samples_leaf': 2,
     'max_features': 'log2', 'bootstrap': True,  'criterion': 'entropy', 'max_samples': 0.8,
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Árboles menos profundos, más sesgo, menos varianza
    {'n_estimators': 200, 'max_depth': 10,  'min_samples_split': 5,  'min_samples_leaf': 2,
     'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'gini',
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Profundidad contenida y hojas mínimas de 3
    {'n_estimators': 300, 'max_depth': 15,  'min_samples_split': 10, 'min_samples_leaf': 3,
     'max_features': 'sqrt', 'bootstrap': True,  'criterion': 'entropy', 'max_samples': 0.7,
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Más árboles, profundidad alta, split moderado
    {'n_estimators': 500, 'max_depth': 20,  'min_samples_split': 6,  'min_samples_leaf': 2,
     'max_features': 'sqrt', 'bootstrap': True,  'criterion': 'gini', 'max_samples': 0.9,
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Log2 + sin bootstrap para máxima varianza-reducción vía todos los datos por árbol
    {'n_estimators': 220, 'max_depth': 18,  'min_samples_split': 4,  'min_samples_leaf': 2,
     'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini',
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Árboles pequeños y rápidos (baseline regularizado)
    {'n_estimators': 120, 'max_depth': 8,   'min_samples_split': 3,  'min_samples_leaf': 2,
     'max_features': 'sqrt', 'bootstrap': True,  'criterion': 'entropy', 'max_samples': 0.75,
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Profundo + hojas mínimas de 4 (reduce overfitting)
    {'n_estimators': 180, 'max_depth': None,'min_samples_split': 5,  'min_samples_leaf': 4,
     'max_features': None, 'bootstrap': True,  'criterion': 'gini', 'max_samples': 0.65,
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Más sesgo (max_depth=6) para datasets ruidosos
    {'n_estimators': 160, 'max_depth': 6,   'min_samples_split': 4,  'min_samples_leaf': 2,
     'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'entropy',
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1},

    # Con límite de hojas para árboles más “simples”
    {'n_estimators': 275, 'max_depth': 14,  'min_samples_split': 5,  'min_samples_leaf': 1,
     'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini',
     'class_weight': 'balanced', 'random_state': 42, 'n_jobs': -1, 'max_leaf_nodes': 2000},
]

print("COMENZANDO A ENTRENAR MODELOS:")

for i, config in enumerate(configuraciones, start=1):

    model = RandomForestClassifier(**config)

    pipeline = Pipeline(
        [ ('preprocesamiento', preprocessor), ('clasificador', model) ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]

    # Evaluación
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy = accuracy_score(y_test, y_pred)
    error_rate = 1 - accuracy
    recall = recall_score(y_test, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    
    model_type = type(pipeline.named_steps['clasificador']).__name__

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    xName = f"run_{timestamp}"
    config_str = json.dumps(config)  # sin indentación, todo en una sola línea

    signature = infer_signature(X_train, pipeline.predict(X_train))
    
    input_example = {
        "age": 18393, "gender": "2", "height": 168, "weight": 62.0, "ap_hi": 110, "ap_lo": 80,
        "cholesterol": 1, "gluc": 1, "smoke": 0, "alco": 0, "active": 1
    }

    with mlflow.start_run(run_name=xName):
        mlflow.log_param("1.Modelo_Usado", model_type)
        mlflow.log_param("2.Transformacion", "category_encoders.TargetEncoder")
        mlflow.log_param("3.Porcen Valid", xPorcen)
        mlflow.log_param("4.Configuracion", config_str)
        mlflow.log_metric("1.Error_Rate", round(error_rate,5))
        mlflow.log_metric("2.Accuracy", round(accuracy,5) )
        mlflow.log_metric("3.ReCall", round(recall,5) )
        mlflow.log_metric("4.Specificity", round(specificity,5) )
        mlflow.sklearn.log_model(
            sk_model = pipeline,
            artifact_path = "modelo_random_forest",
            signature = signature,
            input_example = input_example
        )


COMENZANDO A ENTRENAR MODELOS:
🏃 View run run_20250906_170028 at: http://127.0.0.1:8080/#/experiments/0/runs/e724466506fd4c53a644ef817221ca23
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
🏃 View run run_20250906_170037 at: http://127.0.0.1:8080/#/experiments/0/runs/8c333334b0f148768d39c75a9b0c54e1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
🏃 View run run_20250906_170049 at: http://127.0.0.1:8080/#/experiments/0/runs/0af8f161821d4cbdb7d6cb933027a798
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
🏃 View run run_20250906_170058 at: http://127.0.0.1:8080/#/experiments/0/runs/9f57038592204b629cd7bbf6d0a0a497
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
🏃 View run run_20250906_170106 at: http://127.0.0.1:8080/#/experiments/0/runs/2f95cf75d0a84ab7a757f86eab140287
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
🏃 View run run_20250906_170114 at: http://127.0.0.1:8080/#/experiments/0/runs/76379e64b79a43edb7117c57d6df7d4a
🧪 V

In [9]:

configuraciones = [
    # Regularización estándar, baseline
    {'penalty': 'l2', 'C': 1.0, 'solver': 'lbfgs', 'max_iter': 1000, 'random_state': 42},
    # Regularización fuerte (C pequeño)
    {'penalty': 'l2', 'C': 0.05, 'solver': 'lbfgs', 'max_iter': 2000, 'random_state': 42},
    # Regularización débil (C grande)
    {'penalty': 'l2', 'C': 10.0, 'solver': 'newton-cg', 'max_iter': 1500, 'random_state': 42},
    # L1 con liblinear
    {'penalty': 'l1', 'C': 0.5, 'solver': 'liblinear', 'max_iter': 800, 'random_state': 42},
    # L1 balanceado
    {'penalty': 'l1', 'C': 1.0, 'solver': 'liblinear', 'max_iter': 1000, 'class_weight': 'balanced', 'random_state': 42},
    # Elastic Net 50/50
    {'penalty': 'elasticnet', 'C': 1.0, 'solver': 'saga', 'l1_ratio': 0.5, 'max_iter': 1200, 'random_state': 42},
    # Elastic Net más hacia L1
    {'penalty': 'elasticnet', 'C': 0.7, 'solver': 'saga', 'l1_ratio': 0.7, 'max_iter': 1200, 'random_state': 42},
    # Elastic Net más hacia L2
    {'penalty': 'elasticnet', 'C': 0.7, 'solver': 'saga', 'l1_ratio': 0.3, 'max_iter': 1200, 'random_state': 42},
    # Solver SAG para datasets grandes
    {'penalty': 'l2', 'C': 1.5, 'solver': 'sag', 'max_iter': 1000, 'n_jobs': -1, 'random_state': 42},
    # Solver SAGA con balance de clases
    {'penalty': 'l2', 'C': 0.6, 'solver': 'saga', 'max_iter': 1100, 'class_weight': 'balanced', 'n_jobs': -1, 'random_state': 42},
]

print("COMENZANDO A ENTRENAR MODELOS:")

for i, config in enumerate(configuraciones, start=1):

    model = LogisticRegression(**config)

    pipeline = Pipeline(
        [ ('preprocesamiento', preprocessor), ('clasificador', model) ]
    )

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]

    # Evaluación
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy = accuracy_score(y_test, y_pred)
    error_rate = 1 - accuracy
    recall = recall_score(y_test, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    model_type = type(pipeline.named_steps['clasificador']).__name__
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    xName = f"run_{timestamp}"
    config_str = json.dumps(config)  # sin indentación, todo en una sola línea

    signature = infer_signature(X_train, pipeline.predict(X_train))
    
    input_example = {
        "age": 18393, "gender": "2", "height": 168, "weight": 62.0, "ap_hi": 110, "ap_lo": 80,
        "cholesterol": 1, "gluc": 1, "smoke": 0, "alco": 0, "active": 1
    }

    with mlflow.start_run(run_name=xName):
        mlflow.log_param("1.Modelo_Usado", model_type)
        mlflow.log_param("2.Transformacion", "category_encoders.TargetEncoder")
        mlflow.log_param("3.Porcen Valid", xPorcen)
        mlflow.log_param("4.Configuracion", config_str)
        mlflow.log_metric("1.Error_Rate", round(error_rate,5))
        mlflow.log_metric("2.Accuracy", round(accuracy,5) )
        mlflow.log_metric("3.ReCall", round(recall,5) )
        mlflow.log_metric("4.Specificity", round(specificity,5) )
        mlflow.sklearn.log_model(
            sk_model = pipeline,
            artifact_path = "modelo_random_forest",
            signature = signature,
            input_example = input_example
        )


COMENZANDO A ENTRENAR MODELOS:


C:\Program Files\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


🏃 View run run_20250906_170203 at: http://127.0.0.1:8080/#/experiments/0/runs/a6faf384910f4c83b108f65a47cd1489
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


C:\Program Files\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


🏃 View run run_20250906_170211 at: http://127.0.0.1:8080/#/experiments/0/runs/cacab619ede04762aeb897f601d6c242
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


...

🏃 View run run_20250906_170324 at: http://127.0.0.1:8080/#/experiments/0/runs/e2ef241bec5748ffb16b5820e2a1344a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


C:\Program Files\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


🏃 View run run_20250906_170336 at: http://127.0.0.1:8080/#/experiments/0/runs/706bb60bfada4d2193a1946049cca466
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


C:\Program Files\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


🏃 View run run_20250906_170349 at: http://127.0.0.1:8080/#/experiments/0/runs/b44f65be856146b6b800991b8f1a1108
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
